In [1]:
import torch.nn as nn
import torch
import pandas as pd
import numpy as np
import glob
from astropy.table import Table
%load_ext autoreload
%autoreload 2



In [19]:
from tqdm import tqdm
np.random.seed(99)
cuts = pd.read_csv("ClusterBusters/cuts/cuts.csv")
cuts2 = pd.read_csv("ClusterBusters/cuts/cuts2.csv")
cuts3 = pd.read_csv("ClusterBusters/cuts/cuts3.csv")
cuts = pd.concat((cuts, cuts2), axis=0, ignore_index=True)
cuts = pd.concat((cuts, cuts3), axis=0, ignore_index=True)
streams_test = ['gaia_mock_streams/stream-8495.fits.gz',
 'gaia_mock_streams/stream-2471.fits.gz',
 'gaia_mock_streams/stream-4990.fits.gz',
 'gaia_mock_streams/stream-3775.fits.gz',
 'gaia_mock_streams/stream-5797.fits.gz',
 'gaia_mock_streams/stream-1324.fits.gz',
 'gaia_mock_streams/stream-9420.fits.gz',
 'gaia_mock_streams/stream-3696.fits.gz',
 'gaia_mock_streams/stream-9833.fits.gz']


train =  []
val = []
test = []
streams = []
tr, v, te = 0,0,0
for i in range(len(cuts)):
    file_name = cuts.loc[i].stream_file
    is_test = file_name in streams_test
    is_train = False
    is_val = False
    if not is_test:
        if len(streams) < 30:
            is_train = True
        else:
            is_val = True
    try:
        table = Table.read(file_name, format='fits')

        stream = table.to_pandas()
        stream = stream.query('ra > ' + str(cuts.loc[i].ra_min) + ' & ra < ' + str(cuts.loc[i].ra_max) + ' & dec > ' + str(cuts.loc[i].dec_min) + ' & dec < ' + str(cuts.loc[i].dec_max))
        stream_name = file_name.replace(".fits.gz", "").replace("gaia_mock_streams", "")
        stream["g"] = stream["phot_g_mean_mag"]
        stream["g_bp"] = stream["phot_g_mean_mag"] - stream["phot_bp_mean_mag"]
        stream["g_rp"] = stream["phot_g_mean_mag"] - stream["phot_rp_mean_mag"]
        #print(stream[["g", "g_bp", "g_rp"]])
        #print(stream_name)
        noise = pd.read_csv(f"noise_points/{stream_name}_mul_400_total_noise.csv", index_col=None)
        #print(noise.columns)
        noise["g"] = noise["phot_g_mean_mag"]
        noise["g_bp"] = noise["phot_g_mean_mag"] - noise["phot_bp_mean_mag"]
        noise["g_rp"] = noise["phot_g_mean_mag"] - noise["phot_rp_mean_mag"]
        #print("through")
        stream["is_stream_actual"] = 1
        noise["is_stream_actual"] = 0
        arr = np.zeros(len(stream))
        arr[np.random.choice(np.arange(len(stream)), replace=False, size = int(0.2*len(stream)))] = True
        stream["is_train"] = arr

        n_train = int(np.sum(stream["is_train"]))
        arr = np.zeros(n_train)
        arr[np.random.choice(np.arange(n_train) , replace=False, size = n_train // 2)] = True
        training_stream = stream[stream["is_train"] == 1].copy()

        training_stream["is_reference"] = arr
        k = 1 if is_train else 255
        stream_noise = stream[stream["is_train"] == 0]
        noise_with_stream_noise = pd.concat((noise, stream_noise), sort=False, ignore_index=True)
        final_noise = noise_with_stream_noise.sample(len(training_stream[training_stream.is_train + training_stream.is_reference == 1])*k)
        final_noise["label"] = 0
        training_stream["label"] = 1
        reference_points = training_stream[training_stream.is_reference == 1].copy()
        training_dataset = pd.concat((final_noise, training_stream[training_stream.is_reference == 0].copy()), sort=False, ignore_index = True)
        features = ["ra", "dec", "pmra", "pmdec", "g_bp", "g_rp", "g"]
        reference_points = reference_points[features].to_numpy()
        X = training_dataset[features].to_numpy()
        y = training_dataset["label"].to_numpy()
        if reference_points.shape[0] > 2:
            if is_train:
                tr += 1
                for i in range(X.shape[0]):
                    train.append((X[i].copy(), y[i].copy(), reference_points.copy()))
                streams.append(stream_name)
            elif is_val:
                v += 1
                for i in range(X.shape[0]):
                    val.append((X[i].copy(), y[i].copy(), reference_points.copy()))
                streams.append(stream_name)
            elif is_test:
                te += 1
                for i in range(X.shape[0]):
                    test.append((X[i].copy(), y[i].copy(), reference_points.copy()))
        
    except:
        print(f"File not found {file_name}")
            
        
print(tr, v, te)

File not found gaia_mock_streams/stream-9549.fits.gz
File not found gaia_mock_streams/stream-3649.fits.gz
File not found gaia_mock_streams/stream-5185.fits.gz
File not found gaia_mock_streams/stream-1011.fits.gz
File not found gaia_mock_streams/stream-112.fits.gz
File not found gaia_mock_streams/stream-6739.fits.gz
File not found gaia_mock_streams/stream-4624.fits.gz
File not found gaia_mock_streams/stream-5698.fits.gz
File not found gaia_mock_streams/stream-9391.fits.gz
File not found gaia_mock_streams/stream-6468.fits.gz
File not found gaia_mock_streams/stream-8566.fits.gz
File not found gaia_mock_streams/stream-5204.fits.gz
File not found gaia_mock_streams/stream-3055.fits.gz
File not found gaia_mock_streams/stream-6869.fits.gz
File not found gaia_mock_streams/stream-8947.fits.gz
File not found gaia_mock_streams/stream-8516.fits.gz
File not found gaia_mock_streams/stream-3704.fits.gz
File not found gaia_mock_streams/stream-1333.fits.gz
File not found gaia_mock_streams/stream-2100.fi

In [17]:
print(len(train))
print(len(val))
print(len(test))


4330
430848
46848


In [2]:
import pickle

In [14]:
pickle.dump(train, open("train_set.pkl", "wb"))
pickle.dump(test, open("test_set.pkl", "wb"))
pickle.dump(streams, open("streams.pkl", "wb"))


In [3]:
train = pickle.load(open("train_set.pkl", "rb"))
test = pickle.load(open("test_set.pkl", "rb"))

In [4]:
import models

/scratch/kae358/miniconda2/envs/NLP/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/scratch/kae358/miniconda2/envs/NLP/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/scratch/kae358/miniconda2/envs/NLP/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/scratch/kae358/minico

In [5]:
import sklearn.metrics as metrics
from tqdm import tqdm

In [11]:
import torch.nn.functional as F
import copy
device = "cuda"
num_epochs = 100 # number epoch to train
criterion = torch.nn.CrossEntropyLoss()


def test_model(loader, model):

    correct = 0
    total = 0
    model.eval()
    prediction_tresh = []
    truth = []
    for x, y, ref in loader:
        
        outputs = F.softmax(model(torch.FloatTensor(x).to(device), torch.FloatTensor(ref).to(device)), dim=0)
        predicted = outputs.max(0, keepdim=True)[1]
        truth.append(y)
        prediction_tresh.append(outputs[1].item())
        total += 1
        correct += predicted.item() == y
    
    acc = (100 * correct / total)
    auc = metrics.roc_auc_score(truth, prediction_tresh)
    prec, rec, f1, _ = metrics.precision_recall_fscore_support(truth, np.array(prediction_tresh) > 0.5, average="binary")
    
    return acc, auc, prec, rec, f1

def train_model(model, optimizer, criterion, num_epochs):
    best_val_auc = 0
    model_sd = copy.deepcopy(model.state_dict())
    for epoch in range(num_epochs):
        loss_t = 0
        size = len(train)
        for i, (x, y, ref) in enumerate(np.random.permutation(train)):
            model.train()
            optimizer.zero_grad()
            outputs = model(torch.FloatTensor(x).to(device), torch.FloatTensor(ref).to(device))
            loss = criterion(outputs.unsqueeze(0), torch.LongTensor([y]).to(device))
            loss.backward()
            optimizer.step()
            loss_t += loss.item()
            if i % 2000 == 0:
                print(f"Train loss {i}/{size}: {loss_t/(i+1)}")
        train_acc, t_auc, _, _, _ = test_model(train, model)
        val_acc, val_auc, val_prec, val_rec, val_f1 = test_model(test, model)
        print('Epoch: [{}/{}],  Train acc: {}, Validation Acc: {}, Val auc: {}, Val precision: {}, Val recall: {}, Val f1: {}'.format( 
                       epoch+1, num_epochs,  train_acc, val_acc, val_auc, val_prec, val_rec, val_f1))
        if val_auc > best_val_auc:
            best_val_auc = val_auc
            model_sd = copy.deepcopy(model.state_dict())
        
    return best_val_auc, model_sd
            
    
def hyperparameter_search(n_tries, num_epochs, results_name):
    n_features = 7
    n_hidden = (2,15)
    hidden_size = (5, 100)
    dropout = (0.1, 0.3)
    learning_rates = (-5, -3)
    df = pd.DataFrame(columns=["n_features", "n_hidden", "hidden_size", "dropout", "learning_rate", "val_acc"])
    for i in tqdm(range(n_tries)):
        options = {"n_features": 7,
                   "n_hidden": np.random.randint(n_hidden[0], n_hidden[1] + 1),
                   "hidden_size": np.random.randint(hidden_size[0], hidden_size[1] + 1),
                   "dropout": np.random.uniform(dropout[0], dropout[1])}
        print(options)
        model = models.SeparateEmbeddings(options).to("cuda")
        lr = 10**np.random.uniform(learning_rates[0], learning_rates[1])
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        val_auc, model = train_model(model, optimizer, criterion, num_epochs)
        options["learning_rate"] = lr
        options["val_auc"]  = val_auc
        df = df.append(options, ignore_index=True)
        df.to_csv(results_name)
        
        


In [ ]:
df = pd.read_csv("results_2.csv")
df.sort_values("val_auc", ascending=False)

In [15]:
options = {"n_features": 7, "n_hidden": 9, "hidden_size": 50, "dropout": 0.1}
#model = models.SeparateEmbeddings(options)
#model.to("cuda")
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [18]:
val_auc, model_sd = train_model(model, optimizer, criterion, 10)

Train loss 0/6064: 0.12039840221405029
Train loss 2000/6064: 0.17143114620837374
Train loss 4000/6064: 0.17400520192298613
Train loss 6000/6064: 0.17500173900548943
Epoch: [1/10],  Train acc: 94.11279683377309, Validation Acc: 87.57861635220125, Val auc: 0.9399625981742986, Val precision: 0.8624873609706775, Val recall: 0.8941299790356394, Val f1: 0.8780236747297994
Train loss 0/6064: 0.0011713504791259766
Train loss 2000/6064: 0.17603273444745257
Train loss 4000/6064: 0.16934920619857577
Train loss 6000/6064: 0.1736362089297907
Epoch: [2/10],  Train acc: 94.32717678100263, Validation Acc: 86.0587002096436, Val auc: 0.93210206171521, Val precision: 0.8909090909090909, Val recall: 0.8218029350104822, Val f1: 0.8549618320610688
Train loss 0/6064: 0.06677889823913574
Train loss 2000/6064: 0.1664501957330985
Train loss 4000/6064: 0.16313786542853603
Train loss 6000/6064: 0.16802717727177383
Epoch: [3/10],  Train acc: 94.04683377308707, Validation Acc: 85.42976939203355, Val auc: 0.93252728

In [19]:
val_auc, model_sd_2 = train_model(model, optimizer, criterion, 10)

Train loss 0/6064: 0.0037026405334472656
Train loss 2000/6064: 0.1481317178211708
Train loss 4000/6064: 0.15125344465357515
Train loss 6000/6064: 0.1447324618669852
Epoch: [1/10],  Train acc: 95.05277044854881, Validation Acc: 87.78825995807128, Val auc: 0.9405603241784564, Val precision: 0.8645096056622852, Val recall: 0.8962264150943396, Val f1: 0.8800823468862583
Train loss 0/6064: 0.0010535717010498047
Train loss 2000/6064: 0.15852817967496832
Train loss 4000/6064: 0.15080937300733077
Train loss 6000/6064: 0.15301303744037992
Epoch: [2/10],  Train acc: 94.72295514511873, Validation Acc: 87.99790356394129, Val auc: 0.9502865568784639, Val precision: 0.849566055930569, Val recall: 0.9234800838574424, Val f1: 0.8849824208940232
Train loss 0/6064: 0.03731119632720947
Train loss 2000/6064: 0.1300439710530086
Train loss 4000/6064: 0.14391839028268835
Train loss 6000/6064: 0.15029984057555495
Epoch: [3/10],  Train acc: 95.15171503957784, Validation Acc: 87.68343815513627, Val auc: 0.94636

KeyboardInterrupt: 

,Unnamed: 0,n_features,n_hidden,hidden_size,dropout,learning_rate,val_acc,val_auc
38,38,7.0,9.0,50.0,0.119827,0.000892,NaN,0.942526
57,57,7.0,5.0,42.0,0.166065,0.000556,NaN,0.929572
43,43,7.0,5.0,62.0,0.268502,0.000448,NaN,0.928375
17,17,7.0,6.0,91.0,0.208548,0.000272,NaN,0.927575
4,4,7.0,3.0,53.0,0.109671,0.000706,NaN,0.926771
...,...,...,...,...,...,...,...,...
42,42,7.0,3.0,36.0,0.279322,0.000017,NaN,0.687629
64,64,7.0,4.0,9.0,0.209043,0.000034,NaN,0.685514
50,50,7.0,15.0,11.0,0.250164,0.000019,NaN,0.682559
99,99,7.0,5.0,6.0,0.261245,0.000024,NaN,0.649936


In [ ]:
hyperparameter_search(100, 10, "results_2.csv")

In [20]:
d = {"options": options, "model": model_sd}
torch.save(d, "best_model.p")

In [7]:
d = torch.load("best_model.p")

In [8]:
model = models.SeparateEmbeddings(d["options"])

In [9]:
model.load_state_dict(d["model"])
model.to("cuda")

SeparateEmbeddings(
  (hidden_0): Linear(in_features=14, out_features=50, bias=True)
  (hidden_1): Linear(in_features=50, out_features=50, bias=True)
  (hidden_2): Linear(in_features=50, out_features=50, bias=True)
  (hidden_3): Linear(in_features=50, out_features=50, bias=True)
  (hidden_4): Linear(in_features=50, out_features=50, bias=True)
  (hidden_5): Linear(in_features=50, out_features=50, bias=True)
  (hidden_6): Linear(in_features=50, out_features=50, bias=True)
  (hidden_7): Linear(in_features=50, out_features=50, bias=True)
  (hidden_8): Linear(in_features=50, out_features=50, bias=True)
  (projection): Linear(in_features=50, out_features=2, bias=True)
)

In [12]:
test_model(test, model)

(88.52201257861635,
 0.9540629985628206,
 0.9042904290429042,
 0.8616352201257862,
 0.8824476650563606)